# Experiment
Recreate resutts from emargin article with AdaBoost and AdaBoostV instead of arc-gv.  (add computation of emargin later)

http://colt2008.cs.helsinki.fi/papers/08-Wang.pdf page 11

extend to do cross validation

In [1]:
# Import data sets. 
import ipython_notebook_importer
import DataSet as ds

In [10]:
# All features is normalized to [0, 1]? Sum to 1, or just live in this area?
# THe ioanosphere dataset has an dimension where max is 0, so now i try max of abs

# Five fold cross-validation, use this as test error??!?!?!?
# decision stumps, decision trees with depth 1? Yes.
''' 
    We use a finite set of possible decision
    stumps. Specifically, for each feature we consider 100
    thresholds uniformly distributed on [0, 1]. Therefore the
    size of the base classifier set is 2 × 100 × k, where k
    denotes the number of features.
'''
from sklearn.model_selection import KFold
import ipython_notebook_importer
import AdaBoost 
import AdaBoostV # importing notebooks might take a few seconds. 
import numpy as np

def experiment(name, T=100):
    # Load dataset
    data = ds.DataSet(name, split=False, normalize=True)
    X = data.X
    y = data.y
    
    print(name.upper()+ "  ", "\tTest (CV) Error\t\tMinimum Margin")
    
    kf = KFold(n_splits=5, random_state=42, shuffle=False)
    
    error_ada  = []
    error_adav = []
    error_ada_s = []
    error_adav_s = []
    
    mm_ada     = []
    mm_adav    = []
    mm_ada_s = []
    mm_adav_s = []
    
    error_retrain = []
    error_retrain_v = []
    
    mm_retrain = []
    mm_retrain_v = []
    
    i = 0
    for train_index, test_index in kf.split(X):
        print("\rIteration %i"%i, "\t.", end='')
        i+=1
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # AdaBoost exp
        ada = AdaBoost.AdaBoost()
        ada.fit(X, y, T)
        error_ada.append(1-ada.score(X_test, y_test))
        mm_ada.append(ada.minimal_margin(X_train, y_train))
        print(".", end='')
        
        # AdaBoostV experiment
        ada = AdaBoostV.AdaBoostV()
        ada.fit(X, y, T)
        error_adav.append(1-ada.score(X_test, y_test))
        mm_adav.append(ada.minimal_margin(X_train, y_train))
        print(".", end='')
        
        # AdaBoost; sparsify and select by min margin
        ada = AdaBoost.AdaBoost()
        ada.force_silent = True
        ada.fit(X, y, T) # just to 256 and we won on a single dataset!
        ada = ada.sparsify(X, y, 10, "mm")
        error_ada_s.append(1-ada.score(X_test, y_test))
        mm_ada_s.append(ada.minimal_margin(X_train, y_train))
        print(".", end='')
        
        # AdaBoostV; sparsify, retrain and select by minimal margin
        ada = AdaBoostV.AdaBoostV()
        ada.force_silent = True
        ada.fit(X, y, T) # just to 256 and we won on a single dataset!
        ada = ada.sparsify(X, y, 10)
        error_adav_s.append(1-ada.score(X_test, y_test))
        mm_adav_s.append(ada.minimal_margin(X_train, y_train))
        print(".", end='')
        
        # Retrain Ada
        ada = AdaBoost.AdaBoost()
        ada.force_silent = True
        ada = ada.retrain(X, y, 3, max=T)
        error_retrain.append(1-ada.score(X_test, y_test))
        mm_retrain.append(ada.minimal_margin(X_train, y_train))
        print(".", end='')
        
        # Retrain AdaV
        ada = AdaBoostV.AdaBoostV()
        ada.force_silent = True
        ada = ada.retrain(X, y, 3, max=T)
        error_retrain_v.append(1-ada.score(X_test, y_test))
        mm_retrain_v.append(ada.minimal_margin(X_train, y_train))
        print(".", end='')
    
    print("\tDONE!")
        
    error_ada = np.mean(error_ada)
    error_adav = np.mean(error_adav)
    error_ada_s = np.mean(error_ada_s)
    error_adav_s = np.mean(error_adav_s)
    
    mm_ada  = np.mean(mm_ada)
    mm_adav = np.mean(mm_adav)
    mm_ada_s  = np.mean(mm_ada_s)
    mm_adav_s = np.mean(mm_adav_s)
    
    
    error_retrain = np.mean(error_retrain)
    error_retrain_v = np.mean(error_retrain_v)
    
    mm_retrain = np.mean(mm_retrain)
    mm_retrain_v = np.mean(mm_retrain_v)
    
    print("AdaBoost: \t", round(error_ada,5), "\t\t", round(mm_ada,5))
    print("AdaBoostV: \t", round(error_adav, 5), "\t\t", round(mm_adav,5))
    print("---")
    print("AdaBoost (S): \t", round(error_ada_s,5), "\t\t", round(mm_ada_s,5))
    print("AdaBoostV (S): \t", round(error_adav_s, 5), "\t\t", round(mm_adav_s,5))
    print("Retrain: \t", round(error_retrain, 5), "\t\t", round(mm_retrain,5))
    print("RetrainV: \t", round(error_retrain_v, 5), "\t\t", round(mm_retrain_v,5))
    
    print("")
# repeat train sparsify on 10->100; 

In [11]:
T = 500
for name in ds.DataSet.names: 
    experiment(name, T)

DIABETES   	Test (CV) Error		Minimum Margin
Iteration 3 	........................

AdaBoostV.ipynb:60: RuntimeWarning: divide by zero encountered in double_scalars
  "            self.y_test = y_test\n",
AdaBoostV.ipynb:70: RuntimeWarning: invalid value encountered in true_divide
  "        assert sum(self.sample_weight >= 0) == self.sample_weight.shape[0]\n",
AdaBoostV.ipynb:93: RuntimeWarning: invalid value encountered in true_divide
  "\n",


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!PERFECT!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Iteration 4 	.......	DONE!
AdaBoost: 	 0.17963 		 -0.10846
AdaBoostV: 	 0.19005 		 -0.01519
---
AdaBoost (S): 	 0.17182 		 -0.06301
AdaBoostV (S): 	 0.15884 		 -0.0075
Retrain: 	 0.16535 		 -0.03272
RetrainV: 	 0.15751 		 -0.00435

GERMAN   	Test (CV) Error		Minimum Margin
Iteration 4 	...............................	DONE!
AdaBoost: 	 0.194 		 -0.2124
AdaBoostV: 	 0.234 		 -0.01352
---
AdaBoost (S): 	 0.196 		 -0.12734
AdaBoostV (S): 	 0.217 		 -0.0082
Retrain: 	 0.197 		 -0.05967
RetrainV: 	 0.231 		 -0.00498

IONOSPHERE   	Test (CV) Error		Minimum Margin
Iteration 0 	......!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

AdaBoost.ipynb:85: RuntimeWarning: divide by zero encountered in double_scalars
  "            self.test = True\n",
AdaBoost.ipynb:94: RuntimeWarning: invalid value encountered in true_divide
  "        \"\"\"Train AdaBoost from it's current # hypothesis to T hypothesis. \"\"\"\n",


.

AdaBoostV.ipynb:60: RuntimeWarning: invalid value encountered in log
  "            self.y_test = y_test\n",


AssertionError: nan